In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def mse_function(answer, prediction):
    mse = ((answer - prediction)**2).mean(axis=0)
    for i in range(len(answer.shape)):
        mse = mse.mean(axis=0)
    return mse

Data to test on

In [ ]:
n = 10000
time_span = 5
c = 100
time_step = 0.1
integration_time_step = 0.01
number_timesteps_predict = 2
std = 0

# Load the observations
name = f"{n}_{time_span}_{c}_{str(time_step).replace('.', '')}_{str(integration_time_step).replace('.', '')}_{number_timesteps_predict}_{std}"
test_observations = np.load(f"data_no_std/test_observations_{name}.npy")
test_predictions = np.load(f"data_no_std/test_predictions_{name}.npy")

# Scale the data
maximum = max(np.max(np.abs(test_observations)), np.max(np.abs(test_predictions)))
observations_scaled = test_observations/maximum
predictions_scaled = test_predictions/maximum

# Calculate the mean squared error between the true observation and the imperfect model's prediction
finalmse = mse_function(observations_scaled[:,-1,:] - predictions_scaled[:,-1,:])

In [ ]:
# Variables relating to the data you want to load
n = 10000
time_span = 5
time_step = 0.1
integration_time_step = 0.01
number_timesteps_predict = 1
std = 0

In [ ]:
c_array = np.array([50,100,200,500])
c_vs_loss = np.zeros((3,4))

for i, c in enumerate([50,100,200,500]):
    loaded_model = keras.models.load_model(f'saved_models/timesteps_{number_timesteps_predict}/physical_1_data_{name}.keras')
    prediction = loaded_model.predict(observations_scaled)
    
    if c = 100:
        loaded_model = keras.models.load_model(f'saved_models/timesteps_{number_timesteps_predict}/ML_only_1_data_{name}.keras')
        ml_loss = mse_function(observations_scaled[:,-1,:] - prediction[:,-1,:])